In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator
import os
import PIL
from PIL import Image
from tqdm import tqdm_notebook as tqdm

from shutil import copyfile
import shutil

import pickle

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D, Softmax
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16

from keras.models import load_model

Using TensorFlow backend.


# Lecure avec keras

In [2]:
def sep_train_test(new_paths = ['Split_Data/Airliners/Train', 'Split_Data/Airliners/Test'], path = 'G7_scrapping/Airliners/data', airbus_planes = ['A320', 'A321', 'A350', 'A330'], boeing_planes = ['737', '747', '757', '777'], split_limite = .7):
    os.makedirs(new_paths[0], exist_ok = True)
    os.makedirs(new_paths[1], exist_ok = True)
    
    for fd in os.listdir(new_paths[0]):
        shutil.rmtree(new_paths[0] + '/' + fd, ignore_errors=True)
        
    for fd in os.listdir(new_paths[1]):
        shutil.rmtree(new_paths[1] + '/' + fd, ignore_errors=True)
    
    for plane in airbus_planes:
        os.makedirs(new_paths[0] + '/' + plane)
        os.makedirs(new_paths[1] + '/' + plane)
        
        for pict in os.listdir(path + '/Airbus/' + plane):
            rand = np.random.random()
            if rand <= split_limite:
                copyfile(path + '/Airbus/' + plane + '/' + pict, new_paths[0] + '/' + plane + '/' + pict)
            else:
                copyfile(path + '/Airbus/' + plane + '/' + pict, new_paths[1] + '/' + plane + '/' + pict)
                
    for plane in boeing_planes:
        os.makedirs(new_paths[0] + '/' + plane)
        os.makedirs(new_paths[1] + '/' + plane)
        
        for pict in os.listdir(path + '/Boeing/' + plane):
            rand = np.random.random()
            if rand <= split_limite:
                copyfile(path + '/Boeing/' + plane + '/' + pict, new_paths[0] + '/' + plane + '/' + pict)
            else:
                copyfile(path + '/Boeing/' + plane + '/' + pict, new_paths[1] + '/' + plane + '/' + pict)

In [3]:
sep_train_test(airbus_planes = ['A320', 'A321', 'A350', 'A330', 'A380'], boeing_planes = ['737', '747', '757', '777'])

In [4]:
def convert_format(path = 'Split_Data/Airliners/Test', old_format = 'jpg', new_format = 'png'):
    for file in os.listdir(path):
        print(file)
        for pict in os.listdir(path + '/' + file):
            if pict[-len(old_format):] == old_format:
                im = Image.open(path + '/' + file + '/' + pict)
                im.save(path + '/' + file + '/' + pict[:-len(old_format)] + new_format)
                os.remove(path + '/' + file + '/' + pict)

In [5]:
convert_format(path = 'Split_Data/Airliners/Train', old_format = 'png', new_format = 'jpg')

737
A330
747
777
757
A350
A320
A380
A321


In [6]:
def data_augmentation(train_path, coeff_creation = 2, rotation_range = 10, width_shift_range = .2, height_shift_range = .2, shear_range = .2, zoom_range = .2, horizontal_flip = True, nb_img = 10, save_format = 'jpg'):
    classes = os.listdir(train_path)
    
    datagen = ImageDataGenerator(
           rotation_range=rotation_range,
           width_shift_range=width_shift_range,
           height_shift_range=height_shift_range,
           shear_range=shear_range,
           zoom_range=zoom_range,
           horizontal_flip=horizontal_flip,
           fill_mode='nearest')
    
    for classe in classes:
        picts = os.listdir(train_path + '/' + classe)
        print(classe)
        
        for pict in picts:
            img = Image.open(train_path + '/' + classe + '/' + pict)
            img = np.array(img)
            img = img.reshape((1,) + img.shape)
            i=1
            
            for batch in datagen.flow(img, batch_size=1, save_to_dir=train_path, save_prefix=classe + '/' + classe, save_format=save_format):
                i += 1
                if i > coeff_creation:
                    break  # otherwise the generator would loop indefinitely

In [7]:
data_augmentation('Split_Data/Airliners/Train', coeff_creation = 2)

737
A330
747
777
757
A350
A320
A380
A321


In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [9]:
train_generator = train_datagen.flow_from_directory('Split_Data/Airliners/Train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 17987 images belonging to 9 classes.


In [10]:
train_generator.class_indices

{'737': 0,
 '747': 1,
 '757': 2,
 '777': 3,
 'A320': 4,
 'A321': 5,
 'A330': 6,
 'A350': 7,
 'A380': 8}

In [11]:
test_generator = train_datagen.flow_from_directory('Split_Data/Airliners/Test copie',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 144 images belonging to 9 classes.


# Modèle Lambda

In [116]:
def create_model(input_shape = (256,256,1), nb_couches = 5, nb_neuronnes = 10, kernel = (3, 3), pool = (2, 2), nb_classes = 1):
    model = Sequential()
    
    model.add(InputLayer(input_shape=input_shape))
    
    for i in range(nb_couches):
        model.add(Conv2D(2**(nb_neuronnes + i), kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
                  
        model.add(Conv2D(2**(nb_neuronnes + i), kernel_size=kernel))
        model.add(keras.layers.BatchNormalization())
        model.add(ReLU())
        
        model.add(MaxPooling2D(pool_size=pool))

    model.add(Flatten())

    model.add(Dense(1024))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(512))
    model.add(keras.layers.BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(nb_classes, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

    return model

In [117]:
model = create_model(input_shape = (256,256,3), nb_neuronnes = 4)#, nb_classes = ytrain.shape[1])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 254, 254, 16)      448       
_________________________________________________________________
batch_normalization_82 (Batc (None, 254, 254, 16)      64        
_________________________________________________________________
re_lu_82 (ReLU)              (None, 254, 254, 16)      0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 252, 252, 16)      2320      
_________________________________________________________________
batch_normalization_83 (Batc (None, 252, 252, 16)      64        
_________________________________________________________________
re_lu_83 (ReLU)              (None, 252, 252, 16)      0         
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 126, 126, 16)     

In [118]:
batch_size = 32
epochs = 5

# Lancement de l'entraînement
history = model.fit(data_train, ytrain, batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(data_test, ytest))

Train on 2403 samples, validate on 794 samples
Epoch 1/5
2403/2403 [==============================] - 310s 129ms/step - loss: 0.7931 - accuracy: 0.5734 - val_loss: 0.6665 - val_accuracy: 0.5957
Epoch 2/5
2403/2403 [==============================] - 299s 124ms/step - loss: 0.6023 - accuracy: 0.6787 - val_loss: 0.6893 - val_accuracy: 0.5592
Epoch 3/5
2403/2403 [==============================] - 293s 122ms/step - loss: 0.5261 - accuracy: 0.7399 - val_loss: 0.7679 - val_accuracy: 0.6222
Epoch 4/5
2403/2403 [==============================] - 292s 122ms/step - loss: 0.6316 - accuracy: 0.6633 - val_loss: 10.0990 - val_accuracy: 0.5151
Epoch 5/5
2403/2403 [==============================] - 293s 122ms/step - loss: 0.5147 - accuracy: 0.7511 - val_loss: 1.2679 - val_accuracy: 0.5781


# Transfer learning

In [12]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output

'''x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)'''

#x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# dernière couche que sert a prédire la bonne classe
x = Dense(9)(x)
x = keras.layers.BatchNormalization()(x)
predictions = Softmax()(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [51]:
#model = load_model('model_ext_16.h5')

In [13]:
step_size_train=train_generator.n//train_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=1, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=2,
                   validation_data = test_generator,
                   callbacks=[reduce_lr])

Epoch 1/2
562/562 [==============================] - 4170s 7s/step - loss: 1.3722 - accuracy: 0.5343 - val_loss: 2.4877 - val_accuracy: 0.1597
Epoch 2/2


E0117 16:24:33.009383 140735758197632 ultratb.py:147] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-efdd0b95db4c>", line 13, in <module>
    callbacks=[reduce_lr])
  File "/Users/paul/.local/lib/python3.7/site-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/Users/paul/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1732, in fit_generator
    initial_epoch=initial_epoch)
  File "/Users/paul/.local/lib/python3.7/site-packages/keras/engine/training_generator.py", line 220, in fit_generator
    reset_metrics=False)
  File "/Users/paul/.local/lib/python3.7/site-packages/keras/engine/training.py", line 1514, in train_on_batch
    outputs = self.train_function(ins)
  File "/Users/paul/.local/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py", line 3740, in __call__
    outpu

KeyboardInterrupt: 

In [24]:
def save_model_classes(path_mod: str, mod_name: str, train_generator, model):
    
    shutil.rmtree(path_mod + mod_name, ignore_errors = True)
    os.makedirs(path_mod + mod_name)
    label_map = (train_generator.class_indices)
    with open(path_mod + mod_name + '/' + 'model_' + mod_name + '.pkl', "wb") as f:
        pickle.dump(label_map, f)
    model.save(path_mod + mod_name + '/' + 'model_'+ mod_name + '.h5')


In [25]:
save_model_classes('Models/', '17-01', train_generator, model)